# Проект "Анализ паттернов". Построение алгоритмов выделения паттернов.

In [ ]:
# imports
import pyexcel_xlsx as xl
import pandas as pd
import numpy as np
from itertools import permutations
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
import time

## Проект алгоритма выделения порядково-фиксированных паттернов.

__Основа метода - расстояние Хемминга.__

Описание:

1) подгрузка данных из excel 
    
2) создание дата фрейма регион-партия
    - проверяем файл на ошибки (пропуски, сумма ряда больше 100%, данные другого типа)
3) кодировка регионов (0-1-2)

4) подсчет попарного расстояния Хемминга для каждой пары регионов без перестановок
    - регионы, имеющие нулевое расстояние между собой, относятся к одному паттерну
    
5) визуализация полученного результата
    - график
    - текст

In [ ]:
def encode(df):
    codes = [] # список с кодировками всех регионов (список списков)   
    for i in range(df.shape[0]): # для всех регионов в матрице
        code = ''
        for j in range(df.shape[1] - 1): # для всех партий в регионе
            if df.iloc[i, j] > df.iloc[i, j + 1]:
                code += '2'
            elif df.iloc[i, j] == df.iloc[i, j + 1]:
                code += '0'
            else:
                code += '1'
        codes.append(code)
    return codes

In [ ]:
def error_check_xlsx(df, parties_names, regions_names, f):
    print('Type the path of the xlsx file')
    file_path = input()
    print('Type the path for the output txt file')
    output_file_path = input()
    f = open(output_file_path,'w')
    general_df = xl.get_data(file_path)
    for country_year in general_df.keys():
        print(country_year + ' DATA ERRORS', file = f)
        df = general_df[country_year]
        print(file = f)
        df[0] = df[0][1:] # создание матрицы регион-партия
        parties_names = df[0][0:df[0].index('')]
        regions_names = []
        data = []
        for element in df[1:]:
            regions_names.append(element[0])
            data.append(element[1:])
        pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)
        counter = 0
        df = df.convert_objects(convert_numeric = True) # замена всех нечисловых значений на 0.0
        df.replace(to_replace = np.nan, value = 0, inplace = True)
        counter = 0
        for votes_sum in df.sum(axis = 1):
            if abs(100 - votes_sum) > 0.5:
                print('Sum of region {} votes does not equal 100%, but {}'.format(regions_names[counter], votes_sum),
                     file = f)
            counter += 1
    print('--------------------------------------------------------------', file = f)

In [ ]:
def error_check(df, parties_names, regions_names, f):
    counter = 0
    print('DATA ERRORS', file = f)
    for dtype in df.dtypes:
        if dtype != 'int64':
            print('Party {} contains a non-numeric value'.format(parties_names[counter]), file = f)
        counter += 1
    print()
    df = df.convert_objects(convert_numeric = True) # замена всех нечисловых значений на 0.0
    df.replace(to_replace = np.nan, value = 0, inplace = True)
    counter = 0
    for votes_sum in df.sum(axis = 1):
        if abs(100 - votes_sum) > 1.0:
            print('Sum of region {} votes does not equal 100%, but {}'.format(regions_names[counter], votes_sum),
                 file = f)
        counter += 1
    print('--------------------------------------------------------------', file = f)

In [ ]:
def ordinally_fixed_patterns():
    print('Type the path of the file')
    print('/Users/macbook/Desktop/banks_test.xlsx')
    f = open('ordinally_fixed_patterns','w')
    file_path = input()
    general_df = xl.get_data(file_path)
    for country_year in general_df.keys():
        print(country_year, file = f)
        df = general_df[country_year]
        print('--------------------------------------------------------------', file = f)
        df[0] = df[0][1:] # создание матрицы регион-партия
        parties_names = df[0][0:df[0].index('')]
        regions_names = []
        data = []
        for element in df[1:]:
            regions_names.append(element[0])
            data.append(element[1:])
        pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)
        error_check(pd_df, parties_names, regions_names, f = f) # проверка на ошибки
        codes = encode(pd_df)
        pd_df['code'] = codes
        unique_codes = np.unique(codes) # список уникальных кодировок
        print('{} ordinally-fixed patters have been recognized'.format(len(unique_codes)), file = f)
        print('--------------------------------------------------------------', file = f)
        counter = 1
        for unique_code in unique_codes:
            print('Pattern №{}, encoded as {}, includes following regions:'.format(counter, unique_code), file = f)
            for region in pd_df.loc[pd_df['code'] == unique_code].index:
                print(region, file = f)
            counter += 1
            print('--------------------------------------------------------------', file = f)
        plt.figure(figsize = (12, 8))
        pd.tools.plotting.parallel_coordinates(pd_df[pd_df.columns], 'code')
        plt.legend(pd_df.index)
        plt_name = country_year + '_ordinally_fixed_patterns'
        plt.title(plt_name)
        plt.savefig(plt_name)
        print('==============================================================', file = f)

In [ ]:
ordinally_fixed_patterns()

## Проект алгоритма выделения порядково-инвариантных паттернов.

Инвариантный способ

Описание:

1) подгрузка данных из excel 
       - проверяем файл на ошибки (пропуски, сумма ряда больше 100%, данные другого типа) --> raise error
2) создаем data frame
3) функция кодировки регионов (0-1-2)
4) для каждой возможной перестановки столбцов фрейма
       - кодировка
       - расстояния Хемминга
       - разбиение на паттерны
5) если во всех перестановках гергион относится к одному паттерну, выделяется инвариативный паттерн
6) визуализация полученного результата
        - график
        - текст

In [ ]:
def ordinally_invariant_patterns():
    print('Type the path of the file')
    print('/Users/macbook/Desktop/banks_test.xlsx')
    f = open('ordinally_invariant_patterns','w')
    file_path = input()
    general_df = xl.get_data(file_path)
    for country_year in general_df.keys():
        print(country_year, file = f)
        df = general_df[country_year]
        print('--------------------------------------------------------------', file = f)
        df[0] = df[0][1:] # создание матрицы регион-партия
        parties_names = df[0]#[0:df[0].index('')]
        regions_names = []
        data = []
        for element in df[1:]:
            regions_names.append(element[0])
            data.append(element[1:])
        pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)
        #error_check(pd_df, parties_names, regions_names, f = f) # проверка на ошибки
        transposed_df = pd_df.transpose()
        patterns = []
        for column in transposed_df.columns:
            pattern_to_add = ''
            for i in transposed_df[column].sort_values().index:
                pattern_to_add += i
            patterns.append(pattern_to_add)
        patterns = np.array(patterns)
        unique_patterns = np.unique(patterns)
        regions_names = np.array(regions_names)
        print('{} invariant pattern(s) have been found'.format(unique_patterns.shape[0]), file = f)
        print('--------------------------------------------------------------', file = f)
        counter = 1
        for unique_pattern in unique_patterns:
            print('Invariant Pattern {} includes following regions:'.format(counter), file = f)
            pattern_regions = regions_names[patterns == unique_pattern]
            for region in pattern_regions:
                print(region, file = f)
            counter += 1
            print('--------------------------------------------------------------', file = f)
        print('==============================================================', file = f)
        pd_df['code'] = patterns
        plt.figure(figsize = (25, 20))
        plt.xticks(rotation = 45, fontsize = 25)
        plt.yticks(fontsize = 25)
        pd.tools.plotting.parallel_coordinates(pd_df[pd_df.columns], 'code', colormap = 'gist_rainbow', 
                                              linewidth = 4.0)
        plt.legend(pd_df.index, shadow = True)
        plt_name = country_year + '_ordinally_invariant_patterns'
        plt.title(plt_name, fontsize = 40)
        plt.savefig(plt_name)

In [ ]:
ordinally_invariant_patterns()

## Проект алгоритма выделения диффузионно-инвариантных паттернов.

Описание:

1) построение инваририантных паттернов на основе алгоритма №2.

2) выделение диффузионно-инвариантных паттернов внутри групп инвариантных паттернов
    - диффузионно-инвариантных паттернов будет не меньше, чем инвариантных

In [ ]:
def pair_diffusion_estimation(df): # пусть в df не будет явно задан столбик с кодировкой
    decision = True
    for m in range(df.shape[1] - 1): # для всех столбцов, кроме последнего, содержащего кодировку
        test_df = df
        true_code = encode(df)
        value_1 = test_df.iloc[0, m]
        value_2 = test_df.iloc[1, m]
        test_df.iloc[0, m] = value_2
        test_df.iloc[1, m] = value_1
        codes = encode(test_df)
        if codes != true_code:
            decision = False
        break
    return decision  

In [ ]:
def diffusive_invariant_patterns():
    print('Type the path of the file')
    print('/Users/macbook/Desktop/banks_test.xlsx')
    f = open('diffusive_invariant_patterns','w')
    file_path = input()
    general_df = xl.get_data(file_path)
    for country_year in general_df.keys(): # проходим по каждому листу в excel файле
        df = general_df[country_year]
        print(country_year, file = f)
        print('--------------------------------------------------------------', file = f)
        df[0] = df[0][1:] # создание матрицы регион-партия
        parties_names = df[0]#[0:df[0].index('')]
        regions_names = []
        data = []
        for element in df[1:]:
            regions_names.append(element[0])
            data.append(element[1:])
        pd_df = pd.DataFrame(index = regions_names, columns = parties_names, data = data)
        all_clusters = []
        #error_check(pd_df, parties_names, regions_names, f = f)
        transposed_df = pd_df.transpose()
        sorted_parties = []
        patterns = []
        for column in transposed_df.columns:
            pattern_to_add = ''
            for i in transposed_df[column].sort_values().index:
                pattern_to_add += i
            patterns.append(pattern_to_add)
        patterns = np.array(patterns)
        unique_patterns = np.unique(patterns)
        regions_names = np.array(regions_names)
        num_invariant_patterns = unique_patterns.shape[0]

        diffusion_pattern_counter = 0 # счетчик диффузионно-инвариантных паттернов
        diffusion_patterns_all = [] # лист с индексами объектов для каждого диффузионно-инвариантного паттерна
        inv_indexes = []
        for i in range(num_invariant_patterns): # находим индексы объектов, принадлежащих всем порядково-инвариантным паттернам
            invariant_pattern = unique_patterns[i]
            indexes = []
            for index, pattern in enumerate(patterns):
                if pattern == invariant_pattern:
                    indexes.append(index)
            inv_indexes.append(indexes) # результат - inv_indexes, длина которого равна количеству инвариантных паттернов
        for i in range(len(inv_indexes)): # для каждого инвариантного паттерна ищем его диффузионно-инвариантные подмножества
            diff_patterns_types = [] # лист с типами диффузионно-инвариантных паттернов объектов  конкретного инвариантного паттерна
            for j in range(len(inv_indexes[i])):
                diff_patterns_types.append(np.nan)
            indexes_to_check = inv_indexes[i]
            #print(indexes_to_check)
            inv_cluster = pd_df.iloc[indexes_to_check]
            diff_patterns_types[0] = diff_patterns_types.index(np.nan) # определим тип диффузионного паттерна первого элемента как 0
            diffusion_pattern_counter = 0
            # будем проводить эту операцию пока не определим диффузионно-инвариантный паттерн для каждого объекта инвариантного паттерна
            while np.nan in diff_patterns_types:
                #print(diff_patterns_types)
                for n in range(1, inv_cluster.shape[0]): # возможно улучшение - прохождение только по неизвестным объектам
                    if pair_diffusion_estimation(inv_cluster.iloc[[diffusion_pattern_counter, n]]) == True:
                        diff_patterns_types[n] = diffusion_pattern_counter
                try:
                    diffusion_pattern_counter = diff_patterns_types.index(np.nan)
                except ValueError:
                    pass
            diffusion_patterns_all.append(diff_patterns_types)
        print('{} diffusive-invariant pattern(s) have been found'.format(len(diffusion_patterns_all)), file = f)
        print(file = f)
        diffusion_patterns = np.array(diffusion_patterns_all)
        for i in range(len(diffusion_patterns_all)):
            print('Invariant pattern №{} contains {} diffusion-invariant pattern(s):'.format(i + 1, len(np.unique(diffusion_patterns_all[i]))),
                 file = f)
            print(file = f)
            counter = 0
            for diff_unique in np.unique(diffusion_patterns_all[i]):
                print('Diffusive-invariant pattern №{} in Invariant pattern №{} contains following regions:'.format(counter + 1, i + 1), 
                     file = f)
                counter += 1
                indexes_diff = np.argwhere(diffusion_patterns[i] == diff_unique)
                indexes_diff_list = []
                for index in indexes_diff:
                    indexes_diff_list.append(index[0])
                for region_idx in indexes_diff_list:
                    print(regions_names[inv_indexes[i]][region_idx], file = f)
                print(file = f)
            print('--------------------------------------------------------------', file = f)
        print('==============================================================', file = f)

In [ ]:
start = time.time()
diffusive_invariant_patterns()
end = time.time()
print(end-start)